In [17]:
import damask
import numpy as np
import pyvista as pv
pv.set_jupyter_backend('static')

In [18]:
# adjust to your situation, input file needs to exist
cells = [200,200,200]
file_name_in_1  = 'cubic_texture_equiaxed_damask.dream3d'
file_name_out_1 = f'cubic_texture_equiaxed_damask.vti'

In [19]:
grid1 = damask.Grid.load_DREAM3D(file_name_in_1,feature_IDs=None, cell_data=None, phases='Phases', Euler_angles='EulerAngles', base_group=None)

In [20]:
scaled1 = grid1.scale(cells)

In [21]:
scaled1.save(file_name_out_1)

In [22]:
scaled1 

cells:  200 × 200 × 200
size:   75.0 × 75.0 × 75.0 m³
origin: 0.0   0.0   0.0 m
# materials: 265

In [23]:
config1=damask.ConfigMaterial.load_DREAM3D(file_name_in_1, grain_data=None, cell_data=None, cell_ensemble_data='CellEnsembleData', phases='Phases', Euler_angles='EulerAngles', phase_names='PhaseName', base_group=None)

In [24]:
config1.save()

In [25]:
geom = 'cubic_texture_equiaxed_damask.vti'       # path for geometry file
material_config = 'material.yaml'    # path for material.yaml

In [26]:
v = damask.VTK.load(geom)
material_ID = v.get('material').flatten()

In [27]:
ma = damask.ConfigMaterial.load(material_config)

phases = list(ma['phase'].keys())
info = []

for m in ma['material']:
    c = m['constituents'][0]
    phase = c['phase']
    info.append({'phase':   phase,
                 'phaseID': phases.index(phase),
                 'lattice': ma['phase'][phase]['lattice'],
                 'O':       c['O'],
                })

In [28]:
l = np.array([0,0,1])                            # lab frame direction for IPF

IPF = np.ones((len(material_ID),3),np.uint8)
for i,data in enumerate(info):
    IPF[np.where(material_ID==i)] = \
    np.uint8(damask.Orientation(data['O'],lattice=data['lattice']).IPF_color(l)*255)

In [29]:
v = v.set(f'IPF_{l}',IPF)

p   = np.array([d['phase'] for d in info])
pid = np.array([d['phaseID'] for d in info])
v = v.set('phase',p[material_ID],'phase')
v = v.set('phaseID',pid[material_ID],'phaseID')

v.save('cubic_texture_equiaxed_damask.vti_IPF+phase')

In [30]:
#q = np.array([0,1,1])                            # lab frame direction for IPF
#
#IPFm = np.ones((len(material_ID),3),np.uint8)
#for i,data in enumerate(info):
#    IPFm[np.where(material_ID==i)] = np.uint8(damask.Orientation(data['O'],lattice=data['lattice']).IPF_color(q)*255)

In [31]:
#t = v.set(f'IPFm_{q}',IPF)
#w   = np.array([d['phase'] for d in info])
#wid = np.array([d['phaseID'] for d in info])
#t = v.set('phase',w[material_ID],'phase')
#t = v.set('phaseID',wid[material_ID],'phaseID')
#t.save('cubic_texture_elongated_grains_initial_IPFq+phase')